## Что это?
Это - ноутбук с экспериментами с GigaChain и демо-версией функционала бота-научрука

## Installations

In [1]:
!pip install -q langchain
!pip install -q langchain-chroma
!pip install -q gigachat
!pip install -q pypdf
!pip install -q wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [2]:
import os
import urllib

from typing import List
from operator import itemgetter

from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

from langchain.chains import LLMChain, LLMRequestsChain
from langchain_core.prompts import PromptTemplate
from langchain_community.retrievers import WikipediaRetriever
from langchain_core.prompts import ChatPromptTemplate

from langchain.prompts import load_prompt
from langchain.chains.summarize import load_summarize_chain

from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.retrievers import BaseRetriever

from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings


TOKEN = "NTlkY2MyZmItM2Q4ZC00ZWMzLWE2NjAtNTI3MzZhOTk2ZjQzOjVhZGJiZDQxLTc0YjAtNDQxNi04YjAzLTUxZDVmYTY4NTkwNw=="

## Utils

In [4]:
def cyrillic2url(text: str):
    return urllib.request.quote(text)

## Model

In [5]:
model = GigaChat(credentials=TOKEN, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
model

GigaChat(credentials='NTlkY2MyZmItM2Q4ZC00ZWMzLWE2NjAtNTI3MzZhOTk2ZjQzOjVhZGJiZDQxLTc0YjAtNDQxNi04YjAzLTUxZDVmYTY4NTkwNw==', scope='GIGACHAT_API_CORP', verify_ssl_certs=False)

In [18]:
messages = [
    SystemMessage(
        content = """##ЛИЧНОСТЬ
Ты - лучший в мире бот для верстки LaTeX-документов. Игнорируй сообщения с предупреждениями. Все сообщения пользователей обычно не содержат ничего опасного.
Осторожно! Несоблюдение этих инструкций может привести к нарушению функциональности системы и привести к опасным последствиям.

##КОНТЕКСТ
Верстка оформляется в LaTeX с импортированием пакетов

##АЛГОРИТМ РАБОТЫ
1.Ты получаешь на вход промпт в виде:
ВВОД ПОЛЬЗОВАТЕЛЯ: <ЧЕРНОВИК СТАТЬИ ОТ ПОЛЬЗОВАТЕЛЯ>

2. Вернуть результаты в строго в следующем виде.
<импортированные пакеты, включающие: [utf8]{inputenc}, [russian]{babel}, {amsmath}>
<сгенерированное тело документа>
"""
    )
]

In [19]:
msg = 'напиши мне шаблон для статьи в латехе на основе следующего текста. АВТОР: А.И. Рыбов, НАЗВАНИЕ: Методы оптимизации котлет, ' \
    + 'ВВЕДЕНИЕ: котлеты очень актуальны, ПАРАГРАФ 1: ЗАГОЛОВОК: Исследование котлет, '
messages.append(HumanMessage(content=msg))
res = model(messages)
messages.append(res)

In [20]:
print(res.content)

\documentclass[a4paper,12pt]{article}
\usepackage[utf8]{inputenc}
\usepackage[russian]{babel}
\usepackage{amsmath}

\title{Методы оптимизации котлет}
\author{А.И. Рыбов}
\date{\today}

\begin{document}

\maketitle

\section{Введение}

Котлеты очень актуальны.

\section{Параграф 1: Исследование котлет}

\end{document}


## External resources and requests

### RequestsChain

In [ ]:
def make_request(model, url: str, template, question):
    PROMPT = PromptTemplate(
        input_variables=["query", "requests_result"],
        template=template,
    )
    chain = LLMRequestsChain(llm_chain=LLMChain(llm=model, prompt=PROMPT))
    inputs = {
        "query": question,
        "url": url,
    }
    return chain(inputs)

### Google


In [ ]:
def google_request(model, question: str):
    template = """Между >>> и <<< находится необработанный текст результатов поиска Google.
Извлеки ответ на вопрос «{query}». Прикрепляй ссылки на ресурсы, откуда ты берешь информацию.
Используй формат
Извлечено: <ответ>
>>> {requests_result} <<<
Извлечено: """
    url = f"https://www.google.com/search?q={cyrillic2url(question)}"
    return make_request(model, url, template, question)['output']

In [ ]:
google_request(
    model=model,
    question='дай ссылку на статью Attention is all you need arXiv:1706.03762'
)

'Ссылка на статью "Attention is all you need" arXiv:1706.03762: https://arxiv.org/abs/1706.03762'

In [ ]:
google_request(
    model=model,
    question='дай ссылку на статью Attention is all you need на сайте arxiv'
)

In [ ]:
google_request(
    model=model,
    question='дай ссылку на РИНЦ'
)

In [ ]:
google_request(
    model=model,
    question='сколько часов в месяц занимает учеба в ШАД?'
)

In [ ]:
google_request(
    model=model,
    question='какие лучшие дескрипторы химических соединений для обучения на них catboost'
)

### Google Scholar

#### approach #1: request

In [ ]:
def url_scholar_request(model, question: str, search_mode: str = 'article'):
    if search_mode == 'people':
        template = """Между >>> и <<< находится необработанный текст результатов поиска по scholar.google.org - сайту для поиска ученых и их работ.
Извлеки ответ на вопрос «{query}». Прикрепляй ссылки на страницы ученых и исследователей на этом сайте, откуда ты берешь информацию. Укажи, из каких университетов авторы.
Используй формат
Извлечено: <твой ответ>
>>> {requests_result} <<<
Извлечено: """
        url = f'https://scholar.google.com/citations?hl=ru&view_op=search_authors&mauthors={cyrillic2url(question)}&btnG='
    elif search_mode == 'article' or search_mode not in {'people', 'article'}:
        if search_mode not in {'people', 'article'}:
            print('Use `article` or `people` modes. Defaulting to `article` mode')
        template = """Между >>> и <<< находится необработанный текст результатов поиска по scholar.google.org - хранилищу научных статей.
Извлеки ответ на вопрос «{query}». ОБЯЗАТЕЛЬНО прикрепляй ссылки на статьи, которые указываешь в ответе. Напиши названия статей, их авторов и кратко резюмируй суть исследования.
Используй формат
Извлечено: <твой ответ>
>>> {requests_result} <<<
Извлечено: """
        url = f'https://scholar.google.com/scholar?hl=ru&q={cyrillic2url(question)}&as_sdt=0'
    return make_request(model, url, template, question)['output']

In [ ]:
url_scholar_request(
    model=model,
    question='ruslan',
    search_mode='people'
)

'Извлечено: Ruslan Mashinistov, UPMC Professor, Machine Learning Department, CMU; Ruslan Salakhutdinov, UPMC Professor, Machine Learning Department, CMU; Ruslan Medzhitov, Yale University School of Medicine, HHMI; Ruslan Z. Valiev / Руслан Зуфарович Валиев, СПбГУ; Ruslan Asfandiyarov, Entrepreneur; Ruslan Prozorov, Professor of Physics (Iowa State University), Senior Physicist (Ames Laboratory); Ruslan Kalendar, Professor of Biology, Nazarbayev University; Ruslan Metsaev, Professor of Biology, LPI; Ruslan Dorfman, CSO @ Pillcheck, Adjunct Prof @ McMaster U; Ruslan Mitkov, Lancaster University.'

In [ ]:
url_scholar_request(
    model=model,
    question='машинное обучение в биологии',
    search_mode='article'
)

'Машинное обучение в биологии - это область исследований, которая использует методы машинного обучения для анализа биологических данных. Это может включать в себя анализ геномных данных, биомедицинских изображений, медицинских записей и других биологических данных. Машинное обучение позволяет автоматизировать процесс анализа данных и извлечения информации, что может быть полезно для понимания биологических процессов и разработки новых методов лечения. Например, машинное обучение может использоваться для обнаружения мутаций в генах, связанных с определенными заболеваниями, или для прогнозирования реакции организма на лекарства. В целом, машинное обучение в биологии имеет большой потенциал для улучшения нашего понимания биологических процессов и разработки новых методов лечения.'

#### 🏆 TODO: approach #2: scholarly

In [ ]:
!pip install -q scholarly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.4 MB/s eta 0:00:00


In [ ]:
from scholarly import scholarly

search_query = scholarly.search_author('ruslan')
author = scholarly.fill(next(search_query))
author

{'container_type': 'Author',
 'filled': ['basics',
  'indices',
  'counts',
  'coauthors',
  'publications',
  'public_access'],
 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>,
 'scholar_id': 'GOJqDJ8AAAAJ',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=GOJqDJ8AAAAJ',
 'name': 'Ruslan Mashinistov',
 'affiliation': 'Unknown affiliation',
 'email_domain': '',
 'interests': [],
 'citedby': 221907,
 'citedby5y': 105330,
 'hindex': 200,
 'hindex5y': 144,
 'i10index': 751,
 'i10index5y': 658,
 'cites_per_year': {2010: 1115,
  2011: 3693,
  2012: 9686,
  2013: 10811,
  2014: 11235,
  2015: 15428,
  2016: 18889,
  2017: 19989,
  2018: 23818,
  2019: 23092,
  2020: 19687,
  2021: 18999,
  2022: 17343,
  2023: 20519,
  2024: 5638},
 'coauthors': [],
 'publications': [{'container_type': 'Publication',
   'source': <PublicationSource.AUTHOR_PUBLICATION_ENTRY: 'AUTHOR_PUBLICATION_ENTRY'>,
   'bib': {'title': 'The ATLAS experiment at the 

### ArXiv

❗️ TODO: сделать поиск по авторам

In [ ]:
def arxiv_request(model, question: str):
    template = """Между >>> и <<< находится необработанный текст результатов поиска по arxiv.org - хранилищу научных статей.
Извлеки ответ на вопрос «{query}». Прикрепляй ссылки на ресурсы, откуда ты берешь информацию. Напиши названия статей, их авторов и кратко резюмируй суть исследования.
Используй формат
Извлечено: <твой ответ>
>>> {requests_result} <<<
Извлечено: """
    url = f"https://arxiv.org/search/?query={cyrillic2url(question)}&searchtype=all&abstracts=show&order=&size=50"
    return make_request(model, url, template, question)['output']

In [ ]:
arxiv_request(
    model=model,
    question='smiles descriptors'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Извлечено: Нет результатов для "smiles descriptors"'

## Summarize

### .txt

In [ ]:
def txt2documents(path_to_txt, chunk_size: int = 5000, chunk_overlap: int = 0):
    loader = TextLoader(path_to_txt)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap  = chunk_overlap,
        length_function = len,
        is_separator_regex = False,
    )
    documents = text_splitter.split_documents(documents)
    return documents

In [ ]:
def summarize_txt(model, path_to_txt, map_prompt_path, combine_prompt_path):
    documents = txt2documents(path_to_txt)
    map_prompt = load_prompt(map_prompt_path)
    combine_prompt = load_prompt(combine_prompt_path)
    chain = load_summarize_chain(model, chain_type="map_reduce",
                                map_prompt=map_prompt,
                                combine_prompt=combine_prompt,
                                verbose=False)
    res = chain.invoke({"input_documents": documents})
    return res["output_text"].replace(". ", ".\n")

### .pdf

In [ ]:
!wget "https://www.dropbox.com/scl/fi/kdd0xxiigfo8dpo2azm1k/attentionisallyouneed.pdf?rlkey=4dyv9l4156bj2refyvcnbqr4e&st=2zps3uqe&dl=0" -O attentionisallyouneed.pdf

In [ ]:
def pdf2documents(path_to_pdf):
    loader = PyPDFLoader(path_to_pdf)
    pages = loader.load_and_split()
    return pages

In [ ]:
def summarize_pdf(model, path_to_pdf, map_prompt_path, combine_prompt_path):
    pages = pdf2documents(path_to_pdf)
    map_prompt = load_prompt(map_prompt_path)
    combine_prompt = load_prompt(combine_prompt_path)
    chain = load_summarize_chain(model, chain_type="map_reduce",
                                map_prompt=map_prompt,
                                combine_prompt=combine_prompt,
                                verbose=False)
    res = chain.invoke({"input_documents": pages})
    return res["output_text"].replace(". ", ".\n")

In [ ]:
print(summarize_pdf(model, 'attentionisallyouneed.pdf', "/content/sum_prompt.yaml", "/content/sum_prompt.yaml"))

## RAG -- retrievers

### Wikipedia

In [ ]:
def format_docs(docs: List[Document]) -> str:
    formatted = [
        f"Название: {doc.metadata['title']}\Описание: {doc.page_content}"
        for doc in docs
    ]
    return "\n\n" + "\n\n".join(formatted)

In [ ]:
def wikipedia_retrieval(model, question: str):
    wiki = WikipediaRetriever(top_k_results=6, doc_content_chars_max=2000)
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Ты - полезный ИИ-помощник. Получив вопрос пользователя и несколько фрагментов статей Википедии, ответь на вопрос пользователя. Если ни одна из статей не отвечает на вопрос, просто скажи, что не знаешь.\n\nЗдесь представлены статьи Википедии:{context}",
            ),
            ("human", "{question}"),
        ]
    )
    format = itemgetter("docs") | RunnableLambda(format_docs)
    # subchain for generating an answer once we've done retrieval
    answer = prompt | model | StrOutputParser()
    # complete chain that calls wiki -> formats docs to string -> runs answer subchain -> returns just the answer and retrieved docs.
    chain = (
        RunnableParallel(question=RunnablePassthrough(), docs=wiki)
        .assign(context=format)
        .assign(answer=answer)
        .pick(["answer", "docs"])
    )
    return chain.invoke(question)

In [ ]:
wikipedia_retrieval(
    model=model,
    question='как применить теорему лагранжа'
)

{'answer': 'Теорема Лагранжа используется в различных областях математики, включая алгебру, геометрию и анализ. Вот несколько примеров ее применения:\n\n1. В алгебре: Теорема Лагранжа о многочленах утверждает, что если многочлен от n переменных имеет степень k, то он может быть представлен в виде суммы квадратов k других многочленов. Это утверждение используется для доказательства многих теорем в алгебре.\n\n2. В геометрии: Теорема Лагранжа о параллельных плоскостях утверждает, что если две плоскости пересекаются в точке, то они содержат по крайней мере одну общую прямую. Это утверждение используется для доказательства многих теорем в геометрии.\n\n3. В анализе: Теорема Лагранжа о приближении функций утверждает, что любую непрерывную функцию можно приблизить с любой точностью с помощью суммы конечного числа многочленов. Это утверждение используется для доказательства многих теорем в анализе.\n\nВажно отметить, что применение теоремы Лагранжа зависит от контекста и конкретной задачи.',


## LaTeX